# Investigating Moreton Bay Normalised Differenital Chlorophyll Index (NDCI) phenology
The purpose of this notebook is to download **DEA ARD** from sentinel 2 and/or landsat and calculate NDCI and distangle known high ch eventspotentially leading to poor water quality, algale blooms and impacting local fishing.
This notebook will
1. Collect to the DEA STAC and load libraries.
2. Indentify the region of interest, Moreton bay, and our temporal context
3. Load the relevant satelite data to caclualte NDCI.
4. Extract and aggregate NCDI values over morten bay.
5. Plot the NDCI over time and compare against known inflow data and rainfall.
6. Look for high intesnity NDCI data and try and indentify poor water qu and algal blooms.

For Sentinel-2, NDCI is an index that aims to predict the chlorophyll content in turbid productive waters. It is calculated using the red spectral band Red with the red edge spectral band Red edge 1. This study measures the presence of water through the modified normalised difference water index (MNDWI) and clorophyll-*a* through the normalised difference clorophyll index (NDCI).

MNDWI is calculated from the green and shortwave infrared (SWIR) bands to identify water.
The formula is

$$
\begin{aligned}
\text{MNDWI} = \frac{\text{Green} - \text{SWIR}}{\text{Green} + \text{SWIR}}.
\end{aligned}
$$

When interpreting this index, values greater than 0 indicate water.

NDCI is calculated from the red edge 1 and red bands to identify water.
The formula is

$$
\begin{aligned}
\text{NDCI} = \frac{\text{Red edge 1} - \text{Red}}{\text{Red edge 1} + \text{Red}}.
\end{aligned}
$$

When interpreting this index, high values indicate the presence of clorophyll-*a*.

### Collect to the DEA STAC and load libraries.

In [20]:
# Import required libraries
import pystac_client
import odc.stac 
# Connect to DEA STAC catalog
catalog = pystac_client.Client.open("https://explorer.dea.ga.gov.au/stac")
# Configure the Amazon web service
odc.stac.configure_rio(
    cloud_defaults=True,
    aws={"aws_unsigned": True},
)

### Indentify the region of interest, Moreton bay, and our temporal context

In [3]:
# Set a bounding box
# [xmin, ymin, xmax, ymax] in latitude and longitude
bbox = [153.057702,-27.835773, 153.598474, -27.035502]

# Set a start and end date
start_date = "2021-11-01"
end_date = "2021-12-31"


### Load the relevant satelite data to calculate NDCI.

In [10]:
# Choose products to load
# Here, the Sentinel-2a and Sentinel-2b products are chosen
collections = [
    "ga_s2am_ard_3",
    "ga_s2bm_ard_3",
]
# Build a query with the parameters above
query = catalog.search(
    bbox=bbox,
    collections=collections,
    datetime=f"{start_date}/{end_date}",
)

# Search the STAC catalog for all items matching the query
items = list(query.items())
print(f"Found: {len(items):d} datasets")

NameError: name 'catalog' is not defined

In [8]:
ds = odc.stac.load(
    items,
    bands=["nbart_red", "nbart_green", "nbart_blue"],  
    crs="utm",
    resolution=30,
    groupby="solar_day",
    bbox=bbox,
)


NameError: name 'odc' is not defined

In [ ]:
ds.nbart_red.plot(col="time", robust=True);

In [ ]:
items[0]

In [ ]:
# Set up a filter query
filter_query = "eo:cloud_cover < 20"

# Query with filtering
query = catalog.search(
    bbox=bbox,
    collections=collections,
    datetime=f"{start_date}/{end_date}",
    filter=filter_query,
)

# Load our filtered data
ds_filtered = odc.stac.load(
    query.items(),
    bands=["nbart_red"],
    crs="utm",
    resolution=30,
    groupby="solar_day",
    bbox=bbox,
)

# Plot our filtered data
ds_filtered.nbart_red.plot(col="time", robust=True);

In [ ]:
# Query with sorting
query = catalog.search(
    bbox=bbox,
    collections=collections,
    datetime=f"{start_date}/{end_date}",
    sortby="eo:cloud_cover",
)

# Print out cloud cover values from low to high
[i.properties["eo:cloud_cover"] for i in query.items()]

Extract and aggregate NCDI values over morten bay.

In [ ]:
import datacube
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import xarray as xr

import warnings

warnings.filterwarnings("ignore")

import sys

sys.path.insert(1, "../Tools/")
from dea_tools.datahandling import load_ard
from dea_tools.plotting import rgb, display_map
from dea_tools.bandindices import calculate_indices
from dea_tools.dask import create_local_dask_cluster

# Create local dask cluster to improve data load time
client = create_local_dask_cluster(return_client=True)

In [ ]:
dc = datacube.Datacube(app="Chlorophyll_monitoring")

In [ ]:
# Define the area of interest
latitude = (-27.835773, -27.035502) 
longitude = (153.057702, 153.598474)

# Set the range of dates for the analysis
time = ("2018-07-01", "2019-03-01")

In [ ]:
display_map(x=longitude, y=latitude)